# The main shopping districts and streets in Milan

This is the Notebook realized for the Capstone Project on Data Science.
Main goal: segmenting and clustiring 88 omojeneous area in city of Milan in order to answer is there a fashion district, 

### Business Problem

The city of Milano is famous for fashion shops and restaurants. According to city government the city is divided into 88 areas considered as homogeneous called NIL (Local Identity Zones). Available as open data it is possbile to get for all these aresa all the bulding adresses of the city getting from the pubblic authority the latitude, longitude, number, street or square name for each building. Goal of this projetc is clustering city areas and extend the analysis to the specific adresses of a specific zone in order to classify each area of the 88 NIL zones and each bulding adress into a specific NIL area. 
Lets consider a costumer how deals with fashion and clothes products how wants to open a new store in Milano. The questions are: which is the most fashion areas where the store are concentrating as fashion district and in this district wich are the most appreciated streets squares in the district.
The goal is to answer to an investitor: wich are the more trending fashion districts in Milano and selected one of this district how is possible to get the most trending adresses for fashion shops defined as name of the street or square.
In Miano, if someone is looking to open a fashion store, the project will give some indication.

### Data description

Definition of 88 areas (NIL) of Milano and building adresses are available as open data from Comune di Milano: http://dati.comune.milano.it/dataset. The NIL dataframe will defined as latitude, longitude, name of NIL and NIL ID (unique code of the area). In a first key means analysys, it will be tryed to identify the most fashion NIL area. Another dataframe will be downoloaded from Comune di Milano getting the info on Stree/Square Name, Number, Latitude, Longitude and NIL ID. After scrabing the data the dataset on adresses will be joint to the Areas dataset and for a NIL area classified as fashion zone the classification will be extended to the new dataframe of local adresses in order to get the adresses more fashion shops.

In [ ]:
import pandas as pd
import numpy as np

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Solving environment: - 

## Part I - Importing and scrabing data from the local administration of Milan

Import the data containing the coordinates of the neighborhood of Milano:

Importing the data of streets and squares, and of demographic on neighborhoods:

In [13]:
# The code was removed by Watson Studio for sharing.

Hier an example of the dataframe with the definition of 88 NIL Areas in Milano:

In [14]:
df_data_neigh.head()

,X,Y,FID_1,FID_1_1,ID_NIL,NIL,AreaHA,AreaMQ
0,9.122746,45.520250,0,0,74,SACCO,70.84658,7.084658e+05
1,9.159979,45.526457,1,1,82,COMASINA,92.67346,9.267346e+05
2,9.121394,45.512246,2,2,75,STEPHENSON,56.00979,5.600979e+05
3,9.136299,45.487326,3,3,66,QT 8,102.44374,1.024437e+06
4,9.230270,45.453417,4,4,29,ORTOMERCATO,140.25196,1.402520e+06


In [15]:
df_data_neigh=df_data_neigh.drop(columns=['FID_1', 'FID_1_1'])
df_data_neigh.head()

,X,Y,ID_NIL,NIL,AreaHA,AreaMQ
0,9.122746,45.520250,74,SACCO,70.84658,7.084658e+05
1,9.159979,45.526457,82,COMASINA,92.67346,9.267346e+05
2,9.121394,45.512246,75,STEPHENSON,56.00979,5.600979e+05
3,9.136299,45.487326,66,QT 8,102.44374,1.024437e+06
4,9.230270,45.453417,29,ORTOMERCATO,140.25196,1.402520e+06


In [16]:
df_data_neigh.shape

(88, 6)

In [ ]:
indir='http://dati.comune.milano.it/dataset/5c6519f6-6d26-41c9-b53b-6106e08d1b90/resource/533b4e63-3d78-4bb5-aeb4-6c5f648f7f21/download/ds634_civici_coordinategeografiche_20200107_csv.zip'
df_indir = pd.read_csv(indir, sep=";")

In [19]:
df_indir.head()

,CODICE_VIA,STATO,TIPO,DENOMINAZIONE,DATA_INTITOLAZIONE,ANNO_SOPPRESSIONE,DESCRITTIVO,ANNCSU,OPENSTREETMAP,PROGANNCSU,...,DATA_MODFINE,IDMASTER,PASSOCARRAIO,LIVELLO,CAP,ID_NIL,NIL,LONG_WGS84,LAT_WGS84,Location
0,1,Attiva,Piazza,DUOMO (DEL),17000101.0,0.0,DEL DUOMO,DEL DUOMO,DEL DUOMO,NaN,...,0,231315,0.0,A raso,NaN,NaN,NaN,9.189375,45.464847,"45.4648471777, 9.1893750655"
1,1,Attiva,Piazza,DUOMO (DEL),17000101.0,0.0,DEL DUOMO,DEL DUOMO,DEL DUOMO,NaN,...,0,12,0.0,A raso,20121.0,1.0,DUOMO,9.189250,45.464862,"45.4648624969, 9.18925000834"
2,1,Attiva,Piazza,DUOMO (DEL),17000101.0,0.0,DEL DUOMO,DEL DUOMO,DEL DUOMO,NaN,...,0,231259,0.0,A raso,NaN,NaN,NaN,9.189836,45.464845,"45.4648445278, 9.18983625158"
3,1,Attiva,Piazza,DUOMO (DEL),17000101.0,0.0,DEL DUOMO,DEL DUOMO,DEL DUOMO,NaN,...,0,231322,0.0,A raso,NaN,NaN,NaN,9.189483,45.464847,"45.4648471532, 9.18948344275"
4,1,Attiva,Piazza,DUOMO (DEL),17000101.0,0.0,DEL DUOMO,DEL DUOMO,DEL DUOMO,NaN,...,0,231314,0.0,A raso,NaN,NaN,NaN,9.189314,45.464846,"45.4648464236, 9.18931409415"


In [20]:
df_indir=df_indir.drop(columns=['NIL'], axis=1)
df_indir.head()

,CODICE_VIA,STATO,TIPO,DENOMINAZIONE,DATA_INTITOLAZIONE,ANNO_SOPPRESSIONE,DESCRITTIVO,ANNCSU,OPENSTREETMAP,PROGANNCSU,...,WEBMERC_Y,DATA_MODFINE,IDMASTER,PASSOCARRAIO,LIVELLO,CAP,ID_NIL,LONG_WGS84,LAT_WGS84,Location
0,1,Attiva,Piazza,DUOMO (DEL),17000101.0,0.0,DEL DUOMO,DEL DUOMO,DEL DUOMO,NaN,...,"5695001,4487",0,231315,0.0,A raso,NaN,NaN,9.189375,45.464847,"45.4648471777, 9.1893750655"
1,1,Attiva,Piazza,DUOMO (DEL),17000101.0,0.0,DEL DUOMO,DEL DUOMO,DEL DUOMO,NaN,...,"5695003,8802",0,12,0.0,A raso,20121.0,1.0,9.189250,45.464862,"45.4648624969, 9.18925000834"
2,1,Attiva,Piazza,DUOMO (DEL),17000101.0,0.0,DEL DUOMO,DEL DUOMO,DEL DUOMO,NaN,...,"5695001,0281",0,231259,0.0,A raso,NaN,NaN,9.189836,45.464845,"45.4648445278, 9.18983625158"
3,1,Attiva,Piazza,DUOMO (DEL),17000101.0,0.0,DEL DUOMO,DEL DUOMO,DEL DUOMO,NaN,...,"5695001,4448",0,231322,0.0,A raso,NaN,NaN,9.189483,45.464847,"45.4648471532, 9.18948344275"
4,1,Attiva,Piazza,DUOMO (DEL),17000101.0,0.0,DEL DUOMO,DEL DUOMO,DEL DUOMO,NaN,...,"5695001,329",0,231314,0.0,A raso,NaN,NaN,9.189314,45.464846,"45.4648464236, 9.18931409415"


In [21]:
df_indir_ext=df_indir.join(df_data_neigh.set_index('ID_NIL'), on='ID_NIL')
df_indir_ext.head()

,CODICE_VIA,STATO,TIPO,DENOMINAZIONE,DATA_INTITOLAZIONE,ANNO_SOPPRESSIONE,DESCRITTIVO,ANNCSU,OPENSTREETMAP,PROGANNCSU,...,CAP,ID_NIL,LONG_WGS84,LAT_WGS84,Location,X,Y,NIL,AreaHA,AreaMQ
0,1,Attiva,Piazza,DUOMO (DEL),17000101.0,0.0,DEL DUOMO,DEL DUOMO,DEL DUOMO,NaN,...,NaN,NaN,9.189375,45.464847,"45.4648471777, 9.1893750655",NaN,NaN,NaN,NaN,NaN
1,1,Attiva,Piazza,DUOMO (DEL),17000101.0,0.0,DEL DUOMO,DEL DUOMO,DEL DUOMO,NaN,...,20121.0,1.0,9.189250,45.464862,"45.4648624969, 9.18925000834",9.186948,45.463707,DUOMO,234.17041,2.341704e+06
2,1,Attiva,Piazza,DUOMO (DEL),17000101.0,0.0,DEL DUOMO,DEL DUOMO,DEL DUOMO,NaN,...,NaN,NaN,9.189836,45.464845,"45.4648445278, 9.18983625158",NaN,NaN,NaN,NaN,NaN
3,1,Attiva,Piazza,DUOMO (DEL),17000101.0,0.0,DEL DUOMO,DEL DUOMO,DEL DUOMO,NaN,...,NaN,NaN,9.189483,45.464847,"45.4648471532, 9.18948344275",NaN,NaN,NaN,NaN,NaN
4,1,Attiva,Piazza,DUOMO (DEL),17000101.0,0.0,DEL DUOMO,DEL DUOMO,DEL DUOMO,NaN,...,NaN,NaN,9.189314,45.464846,"45.4648464236, 9.18931409415",NaN,NaN,NaN,NaN,NaN


In [22]:
df_indir_ext.shape

(62793, 41)

In [23]:
df_focus=df_indir_ext[df_indir_ext['NIL'].str.contains('BUENOS')==True]

In [24]:
df_focus.head()

,CODICE_VIA,STATO,TIPO,DENOMINAZIONE,DATA_INTITOLAZIONE,ANNO_SOPPRESSIONE,DESCRITTIVO,ANNCSU,OPENSTREETMAP,PROGANNCSU,...,CAP,ID_NIL,LONG_WGS84,LAT_WGS84,Location,X,Y,NIL,AreaHA,AreaMQ
14939,2095,Attiva,Via,AMEDEO D'AOSTA,0.0,0.0,AMEDEO D'AOSTA,AMEDEO D'AOSTA,AMEDEO D'AOSTA,NaN,...,20129.0,21.0,9.217854,45.475205,"45.4752047823, 9.21785389052",9.214489,45.477025,BUENOS AIRES - VENEZIA,287.75423,2.877542e+06
14940,2095,Attiva,Via,AMEDEO D'AOSTA,0.0,0.0,AMEDEO D'AOSTA,AMEDEO D'AOSTA,AMEDEO D'AOSTA,NaN,...,20129.0,21.0,9.218323,45.474960,"45.4749601462, 9.2183230276",9.214489,45.477025,BUENOS AIRES - VENEZIA,287.75423,2.877542e+06
14941,2095,Attiva,Via,AMEDEO D'AOSTA,0.0,0.0,AMEDEO D'AOSTA,AMEDEO D'AOSTA,AMEDEO D'AOSTA,NaN,...,20129.0,21.0,9.218348,45.475772,"45.4757722368, 9.21834803759",9.214489,45.477025,BUENOS AIRES - VENEZIA,287.75423,2.877542e+06
14942,2095,Attiva,Via,AMEDEO D'AOSTA,0.0,0.0,AMEDEO D'AOSTA,AMEDEO D'AOSTA,AMEDEO D'AOSTA,NaN,...,20129.0,21.0,9.218318,45.475249,"45.4752486804, 9.21831817759",9.214489,45.477025,BUENOS AIRES - VENEZIA,287.75423,2.877542e+06
14943,2095,Attiva,Via,AMEDEO D'AOSTA,0.0,0.0,AMEDEO D'AOSTA,AMEDEO D'AOSTA,AMEDEO D'AOSTA,NaN,...,20129.0,21.0,9.217720,45.475205,"45.4752050374, 9.21771972354",9.214489,45.477025,BUENOS AIRES - VENEZIA,287.75423,2.877542e+06


In [25]:
ADR=df_focus['TIPO']+" "+df_focus['DESCRITTIVO']+" "+df_focus['NUMERO'].astype(str)
df_focus.insert(2, "ADRESS", ADR, True)

In [26]:
df_focus.head()

,CODICE_VIA,STATO,ADRESS,TIPO,DENOMINAZIONE,DATA_INTITOLAZIONE,ANNO_SOPPRESSIONE,DESCRITTIVO,ANNCSU,OPENSTREETMAP,...,CAP,ID_NIL,LONG_WGS84,LAT_WGS84,Location,X,Y,NIL,AreaHA,AreaMQ
14939,2095,Attiva,Via AMEDEO D'AOSTA 11,Via,AMEDEO D'AOSTA,0.0,0.0,AMEDEO D'AOSTA,AMEDEO D'AOSTA,AMEDEO D'AOSTA,...,20129.0,21.0,9.217854,45.475205,"45.4752047823, 9.21785389052",9.214489,45.477025,BUENOS AIRES - VENEZIA,287.75423,2.877542e+06
14940,2095,Attiva,Via AMEDEO D'AOSTA 6,Via,AMEDEO D'AOSTA,0.0,0.0,AMEDEO D'AOSTA,AMEDEO D'AOSTA,AMEDEO D'AOSTA,...,20129.0,21.0,9.218323,45.474960,"45.4749601462, 9.2183230276",9.214489,45.477025,BUENOS AIRES - VENEZIA,287.75423,2.877542e+06
14941,2095,Attiva,Via AMEDEO D'AOSTA 15,Via,AMEDEO D'AOSTA,0.0,0.0,AMEDEO D'AOSTA,AMEDEO D'AOSTA,AMEDEO D'AOSTA,...,20129.0,21.0,9.218348,45.475772,"45.4757722368, 9.21834803759",9.214489,45.477025,BUENOS AIRES - VENEZIA,287.75423,2.877542e+06
14942,2095,Attiva,Via AMEDEO D'AOSTA 8,Via,AMEDEO D'AOSTA,0.0,0.0,AMEDEO D'AOSTA,AMEDEO D'AOSTA,AMEDEO D'AOSTA,...,20129.0,21.0,9.218318,45.475249,"45.4752486804, 9.21831817759",9.214489,45.477025,BUENOS AIRES - VENEZIA,287.75423,2.877542e+06
14943,2095,Attiva,Via AMEDEO D'AOSTA 11,Via,AMEDEO D'AOSTA,0.0,0.0,AMEDEO D'AOSTA,AMEDEO D'AOSTA,AMEDEO D'AOSTA,...,20129.0,21.0,9.217720,45.475205,"45.4752050374, 9.21771972354",9.214489,45.477025,BUENOS AIRES - VENEZIA,287.75423,2.877542e+06


In [27]:
df_focus=df_focus[['ADRESS', 'NIL','LONG_WGS84','LAT_WGS84']]
df_focus.drop_duplicates(subset ="ADRESS", 
                     keep = False, inplace = True) 


Hier an example of the dataframe combined between NIL definition and Adresses in the Area:

In [28]:
df_focus.head()

,ADRESS,NIL,LONG_WGS84,LAT_WGS84
14940,Via AMEDEO D'AOSTA 6,BUENOS AIRES - VENEZIA,9.218323,45.474960
14942,Via AMEDEO D'AOSTA 8,BUENOS AIRES - VENEZIA,9.218318,45.475249
14951,Via AMEDEO D'AOSTA 13,BUENOS AIRES - VENEZIA,9.218119,45.475452
14952,Via AMEDEO D'AOSTA 10,BUENOS AIRES - VENEZIA,9.218410,45.475447
14953,Via AMEDEO D'AOSTA 2,BUENOS AIRES - VENEZIA,9.218759,45.474509


In [29]:
df_focus=df_focus.sample(100)